In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import hypot

import constants
import functions5 as functions

%matplotlib inline

In [2]:
all_static_data = functions.load_static_sensors_calibrated()
all_static_data = functions.transform_to_grid_coordinates(all_static_data)
mobile_sensor_data = functions.load_mobile_sensors()
mobile_sensor_data = functions.transform_to_grid_coordinates(mobile_sensor_data)

In [3]:
closest_grid=[[-1 for j in range(20)] for i in range(20)]

for i in range(20):
    for j in range(20):
        distance = 400
        for sensor, coords in constants.gridStaticCoords.items():
            distance1 = hypot(i-coords[0], j-coords[1])
            if distance1 < distance:
                closest_grid[i][j] = coords
                distance = distance1

In [13]:
start_time = '2018-07-03 00:00:00'
end_time = '2018-07-07 00:00:00'
window = 15

start_window = start_time
end_window = (datetime.strptime(start_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=window)).strftime("%Y-%m-%d %H:%M:%S")

all_to_test_list = []
while end_window <= end_time:
    to_test = mobile_sensor_data[(mobile_sensor_data['Timestamp'] > start_window) & (mobile_sensor_data['Timestamp'] < end_window)]
    to_test = to_test.groupby(['lat_grid','long_grid']).mean()
    to_test['Timestamp'] = start_window
    to_test.reset_index(level=to_test.index.names, inplace=True)
    all_to_test_list.append(to_test)


    start_window = (datetime.strptime(start_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=window)).strftime("%Y-%m-%d %H:%M:%S")
    end_window = (datetime.strptime(end_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=window)).strftime("%Y-%m-%d %H:%M:%S")

all_to_test = pd.concat(all_to_test_list, axis = 0, ignore_index = False)

for index, row in all_to_test.iterrows():
    end_window_static = row['Timestamp']
    start_window_static = (datetime.strptime(row['Timestamp'], '%Y-%m-%d %H:%M:%S') - timedelta(minutes=window)).strftime("%Y-%m-%d %H:%M:%S")
    
    # get the value
    stationary_sensor_coords = closest_grid[int(row['lat_grid'])][int(row['long_grid'])]
    stationary_sensor_readings= all_static_data[(all_static_data['Timestamp'] < end_window_static)&(all_static_data['Timestamp'] > start_window_static)&(all_static_data['lat_grid'] == stationary_sensor_coords[0])&(all_static_data['long_grid'] == stationary_sensor_coords[1])]
    all_to_test['baseline_pred'] = stationary_sensor_readings['PM2.5'].mean()    
    
# calculate mae e mse dos erros
print (mean_absolute_error(all_to_test['PM2.5'], all_to_test['baseline_pred']))
print (mean_squared_error(all_to_test['PM2.5'], all_to_test['baseline_pred']))

2.8127296848059404
14.975724728920813
